# Conceptual document similarity with word embeddings

There are a lot of different ways to compare the similarity of words and documents. Sometimes it's technical, sometimes it's a little more theoretical. In this section we'll look at **comparing the concepts** of multiple documents instead of **just comparing the words**.

<p class="reading-options">
  <a class="btn" href="/text-analysis/document-similarity-using-word-embeddings">
    <i class="fa fa-sm fa-book"></i>
    Read online
  </a>
  <a class="btn" href="/text-analysis/notebooks/Document similarity using word embeddings.ipynb">
    <i class="fa fa-sm fa-download"></i>
    Download notebook
  </a>
  <a class="btn" href="https://colab.research.google.com/github/littlecolumns/ds4j-notebooks/blob/master/text-analysis/notebooks/Document similarity using word embeddings.ipynb" target="_new">
    <i class="fa fa-sm fa-laptop"></i>
    Interactive version
  </a>
</p>

## The issue

Computers are very, very, very particular about what counts as "the same." For example, `cat` is the same as `cat` but the _capitalized_ version of `cat` is _not_ the same as the small version.

In [7]:
print('Is cat the same as cat?', "cat" == "cat")
print('Is CAT the same as cat?', "CAT" == "cat")

Is cat the same as cat? True
Is CAT the same as cat? False


If we're looking at similarity **just using the letters in the words**, we can do things like rearrange letters, or use [edit distance](https://en.wikipedia.org/wiki/Edit_distance) for partial matches or all sorts of things in order to make "better" comparisons.

This pickiness ends up being a bit of a pain when comparing documents to se how similar they are to one another. No matter what we do, no amount of counting words is going to get us the _concepts_ in the piece, or the idea of what it's _really about_.

Let's start by taking a look at "normal" document similarity, then compare that approach with using word embeddings on the exact same sentences. **Here are the sentences we'll be using.**

In [107]:
sentences = [
    'Molly ate a donut',
    'Molly ate a fish',
    'Jen consumed a carp',
    'Lenny fears the lions'
]

print('\n'.join(sentences))

Molly ate a donut
Molly ate a fish
Jen consumed a carp
Lenny fears the lions


**Which pair do you think is the most similar?** Two of them are Molly eating something, while two of them are women eating fish. Lenny is... definitely an outlier. While you weigh the options, let's get analyzing!

## Word counting

If you read our piece on n-gram document similarity, you know that there's a lot that goes into whether two documents are similar or not. And if you didn't, no big deal! It totally doesn't matter.

To judge similarity between these sentences, we're going to use a `TfidfVectorizer` from scikit-learn. Less common words are stressed, more common words are more important, and words in long sentences mean less than words in short sentences. **We're counting words in a fancy way.**

In [94]:
from sklearn.feature_extraction.text import CountVectorizer 

vectorizer = CountVectorizer(binary=True)
matrix = vectorizer.fit_transform(sentences)
counts = pd.DataFrame(
    matrix.toarray(),
    index=sentences,
    columns=vectorizer.get_feature_names())
counts

,ate,carp,consumed,donut,fears,fish,jen,lenny,lions,molly,the
Molly ate a donut,1,0,0,1,0,0,0,0,0,1,0
Molly ate a fish,1,0,0,0,0,1,0,0,0,1,0
Jen consumed a carp,0,1,1,0,0,0,1,0,0,0,0
Lenny fears the lions,0,0,0,0,1,0,0,1,1,0,1


We'll be measuring similarity via [cosine similarity](https://www.machinelearningplus.com/nlp/cosine-similarity/), a standard measure of similarity in natural language processing. It's similar to how we might look at a graph with points at `(0,0)` and `(2,3)` and measure the distance between them - just a bit more complicated.

In [103]:
# Compute the similarities using the word counts
similarities = cosine_similarity(matrix)

# Make a fancy colored dataframe about it
pd.DataFrame(similarities,
             index=sentences,
             columns=sentences) \
            .style \
            .background_gradient(axis=None)

,Molly ate a donut,Molly ate a fish,Jen consumed a carp,Lenny fears the lions
Molly ate a donut,1,0.666667,0,0
Molly ate a fish,0.666667,1,0,0
Jen consumed a carp,0,0,1,0
Lenny fears the lions,0,0,0,1


Document similarity is on a scale of zero to one, with zero being completely dissimilar and one being an exact match. Each sentence has a `1` when compared to itself - they're totally equal!

* "Molly ate a donut" and "Molly ate a fish" are both pretty similar - over half - since there's only one word that's different between the two.
* "Jen consumed a carp" only has the nigh-useless "a" in common with them, so it has a similarity score of 0 to both of the others.
* Lenny's sentence also has no shared words with anything else. Nor any topics in common, although it doesn't matter right now.

In our brains, though, _consumed_ means just about the same thing as _ate_. And a carp is a kind of fish, right? **If only there were some way of teaching a computer the meaning behind words!**

## Word embeddings

Word embeddings are a step up from just counting words. Word embeddings give words _meaning_ to computers, teaching it that puppies are kind of like kittens, kittens are like cats, and shoes are very very different from all of those animals.

We're going to be using the [spaCy word embeddings](https://spacy.io/usage/vectors-similarity/). Each word comes with a **300-dimension vector** that expresses things like how catlike the word is, whether you can wear it, if it's something people do during a basketball game (not those exactly, but the same idea). Think of it like 300 different scores for each word, all in different categories.

If you're running this notebook yourself, it'll take a while to load it in. It's a lot of data!

In [96]:
import spacy

nlp = spacy.load("en_core_web_md")

For example, let's check out the 300 dimensions of facts and feelings that spaCy knows about the word `cat`.

In [97]:
nlp('cat').vector

array([-0.15067  , -0.024468 , -0.23368  , -0.23378  , -0.18382  ,
        0.32711  , -0.22084  , -0.28777  ,  0.12759  ,  1.1656   ,
       -0.64163  , -0.098455 , -0.62397  ,  0.010431 , -0.25653  ,
        0.31799  ,  0.037779 ,  1.1904   , -0.17714  , -0.2595   ,
       -0.31461  ,  0.038825 , -0.15713  , -0.13484  ,  0.36936  ,
       -0.30562  , -0.40619  , -0.38965  ,  0.3686   ,  0.013963 ,
       -0.6895   ,  0.004066 , -0.1367   ,  0.32564  ,  0.24688  ,
       -0.14011  ,  0.53889  , -0.80441  , -0.1777   , -0.12922  ,
        0.16303  ,  0.14917  , -0.068429 , -0.33922  ,  0.18495  ,
       -0.082544 , -0.46892  ,  0.39581  , -0.13742  , -0.35132  ,
        0.22223  , -0.144    , -0.048287 ,  0.3379   , -0.31916  ,
        0.20526  ,  0.098624 , -0.23877  ,  0.045338 ,  0.43941  ,
        0.030385 , -0.013821 , -0.093273 , -0.18178  ,  0.19438  ,
       -0.3782   ,  0.70144  ,  0.16236  ,  0.0059111,  0.024898 ,
       -0.13613  , -0.11425  , -0.31598  , -0.14209  ,  0.0281

We have **absolutely no idea what it means**, but sure, okay! If you don't trust it means something, go pop back to the original word embeddings section where we make some nice charts and graphs about puppies and kittens. They'll hopefully convince you!

In the same way that each word has a 300-dimension vector, **entire sentences are just combinations of the words inside.** We can feed spaCy a sentence and it'll spit out another 300 numbers, just like it did for `cat`.

In [98]:
nlp('Some people have never eaten a taco').vector

array([-9.35721472e-02, -6.58728555e-02, -1.09004125e-01, -7.26737157e-02,
       -3.09685934e-02,  2.14037150e-01, -8.41091350e-02, -1.10409282e-01,
        1.90102849e-02,  2.27796984e+00, -1.21881999e-01,  1.30912876e-02,
        1.47405550e-01, -9.49697122e-02, -1.73579574e-01, -8.44508559e-02,
       -7.09174722e-02,  7.88924217e-01, -1.76804587e-01,  9.65175703e-02,
        4.83218543e-02, -1.74057707e-01, -5.80976345e-02,  1.79618560e-02,
       -2.44358610e-02, -1.17278710e-01, -1.59582719e-01, -2.81308684e-02,
        2.24118426e-01, -1.89423576e-01,  2.19992865e-02,  6.82982877e-02,
       -3.31260003e-02, -1.45007715e-01,  5.38914166e-02,  6.10024370e-02,
        5.77065684e-02, -8.49509910e-02, -1.44049406e-01,  1.53504863e-01,
       -7.40417242e-02,  9.36794057e-02,  1.16259269e-01,  4.04221453e-02,
        4.16349284e-02,  2.40260854e-01, -2.14091435e-01,  8.17197189e-02,
        2.16164559e-01,  9.44399908e-02, -4.08502042e-01,  8.20949897e-02,
        9.62229446e-02, -

Again: no idea what it means, but **we'll trust it works** (and we'll see later on!).

In order to find the similarity of each of our sentences, we'll need to conver them each into vectors.

In [109]:
# We aren't printing this because it's 3 * 300 = 900 numbers
vectors = [nlp(sentence).vector for sentence in sentences]

# Print out some notes about it
print("We have", len(vectors), "different vectors")
print("And the first one has", len(vectors[0]), "measurements")
print("And the second one has", len(vectors[1]), "measurements")
print("And the third one has", len(vectors[2]), "measurements")
print("And the fourth one has", len(vectors[3]), "measurements")

We have 4 different vectors
And the first one has 300 measurements
And the second one has 300 measurements
And the third one has 300 measurements
And the fourth one has 300 measurements


It might be useful to compare these 300 measurements-per-sentence to what we were doing before. If we look back to when we were doing similarity by counting words, we only had **eleven measurements for each sentence: one count for every unique word.**

In [110]:
counts

,ate,carp,consumed,donut,fears,fish,jen,lenny,lions,molly,the
Molly ate a donut,1,0,0,1,0,0,0,0,0,1,0
Molly ate a fish,1,0,0,0,0,1,0,0,0,1,0
Jen consumed a carp,0,1,1,0,0,0,1,0,0,0,0
Lenny fears the lions,0,0,0,0,1,0,0,1,1,0,1


Those extra 289 dimensions that word embeddings bring are packed with **concepts** instead of just **word comparisons**. Fishiness, eatinginess, catlikeness, it's all in there! (kind of, maybe, somewhat)

We'll use the same similarity measure as before, cosine similarity. It takes our numeric description of each sentence and checks the distances between them.

In [111]:
# Compute similarities
similarities = cosine_similarity(vectors)

# Turn into a dataframe
pd.DataFrame(similarities,
            index=sentences,
            columns=sentences) \
            .style \
            .background_gradient(axis=None)

,Molly ate a donut,Molly ate a fish,Jen consumed a carp,Lenny fears the lions
Molly ate a donut,1,0.86394,0.697743,0.346385
Molly ate a fish,0.86394,1,0.860143,0.442142
Jen consumed a carp,0.697743,0.860143,1,0.491809
Lenny fears the lions,0.346385,0.442142,0.491809,1


**And there we go!** The first thing to notice is that _none of these sentences are totally dissimilar_. Last time we had multiple zeroes - this time there are none at all, not even with Lenny's weird adventures.

Next up and most importantly: "Molly ate a donut" and "Molly ate a fish" are still very similar with a choice in the 80's, but it turns out "Molly ate a fish" is also _almost just as similar to "Jen consumed a carp"!_

* `0.86394` eating a donut vs. eating a fish
* `0.86014` eating a fish vs. consuming a carp

Thanks to spaCy's word embeddings understanding the concepts behind the words, the computer was able to measure the similarity between "ate" and "consumed," along with "carp" and "fish." As a result we got magically high scores despite the important words not matching exactly!

We might also note that the "fish" and "carp" sentences are slightly more similar to Lenny's lions than the donut sentence. Might it be because _those three involve animals?_

## Gotchas

While this is definitely *exciting*, using word embeddings for similarity isn't without its limits. The biggest issue is what *we* think is similar compared to what the *computer* thinks is similar. Because of the magic of word embeddings, we can't exactly measure what the computer knows!

In [105]:
# Here are our sentences
sentences = [
    'Veronica hates mustard',
    'Veronica loves ketchup',
    'Joseph hates ketchup',
]

In [106]:
# Turn into vectors
vectors = [nlp(sentence).vector for sentence in sentences]

# Compute similarities
similarities = cosine_similarity(vectors)

# Turn into a dataframe
pd.DataFrame(similarities,
            index=sentences,
            columns=sentences) \
            .style \
            .background_gradient(axis=None)

,Veronica hates mustard,Veronica loves ketchup,Joseph hates ketchup
Veronica hates mustard,1,0.866612,0.824512
Veronica loves ketchup,0.866612,1,0.823573
Joseph hates ketchup,0.824512,0.823573,1


So what's more similar, two people hating condiments or two people having emotional feelings about ketchup? **This is a terrible example, but you'll survive until I figure out a better one.**

## Review

In this section, we looked at how **word embeddings** can allow more in-depth comparisons between texts than just plain word counting. Word embeddings allow the computer to understance **nuance** and **conceptual similarity** as opposed to just word-by-word counting.

There are shortcomings to both word counting and word embeddings, but depending on whether you're looking for exact matches or a more conceptual pairing, **either one can be the correct choice**.

## Discussion topics

I'm looking for news websites that republish works without attribution, giving them slight edits. Would I search for them using word counts or word embeddings? What would be the benefit compared to doing this versus reading them manually?

If I wanted to compare the platforms of two politicians, should I use word counts or word embeddings? What would be the benefit compared to doing this versus reading them manually?

If I wanted to compare the platforms of hundreds of local races, should I use word counts or word embeddings? What would be the benefit compared to doing this versus reading them manually? What about using something like [topic modeling](/text-analysis/topic-modeling-and-clustering) instead?
